#  CMS E2E Classification – Evaluation Test (GSoC 2025)
This notebook presents an End-to-End deep learning pipeline to classify CMS detector images of photons and electrons using a custom ResNet-15 model.

- **Dataset**: 32x32 CMS image crops with 2 channels (energy, time)
- **Model**: ResNet-15-like CNN
- **Goal**: Achieve high classification accuracy distinguishing photons vs electrons

##  Load the CMS HDF5 Dataset

We load the raw image crops for photons and electrons from the provided HDF5 files.  
Each file contains:
- `X`: CMS image tensors with shape `(N, 2, 32, 32)`  
- `y`: Corresponding labels (0 = Photon, 1 = Electron)


In [1]:
import h5py
import numpy as np

# Load HDF5 files for photons and electrons from Kaggle input path
photons_file = h5py.File("/kaggle/input/data-data/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5", "r")
electrons_file = h5py.File("/kaggle/input/data-data/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5", "r")

# Print available keys to understand the structure of the files
print("Photon keys:", list(photons_file.keys()))
print("Electron keys:", list(electrons_file.keys()))

Photon keys: ['X', 'y']
Electron keys: ['X', 'y']


##  Prepare the CMS Dataset for Training

We load the image tensors (`X`) and labels (`y`) for both photons and electrons.  
Then we:
- Explicitly assign labels: `0` for photons, `1` for electrons
- Combine both datasets into a single array for training

This gives us a balanced dataset with equal representation from both classes.


In [2]:
# Load the image arrays and labels from HDF5 files
X_photons = np.array(photons_file["X"])  # Shape: (N_photons, 2, 32, 32)
y_photons = np.array(photons_file["y"])  # Initially may contain default values

X_electrons = np.array(electrons_file["X"])
y_electrons = np.array(electrons_file["y"])

# Explicitly assign labels for binary classification
y_photons[:] = 0  # Class 0: Photon
y_electrons[:] = 1  # Class 1: Electron

# Combine photon and electron samples
X = np.concatenate([X_photons, X_electrons], axis=0)
y = np.concatenate([y_photons, y_electrons], axis=0)

# Confirm the final dataset shape and label distribution
print(f"Total samples: {X.shape[0]}, Labels: {np.unique(y, return_counts=True)}")

Total samples: 498000, Labels: (array([0., 1.], dtype=float32), array([249000, 249000]))


##  Data Preparation and Splitting

Now that we have all the CMS data loaded, we:
- Convert the NumPy arrays into PyTorch tensors
- Split the data into training (80%) and testing (20%) sets using stratified sampling
- Wrap the data into `TensorDataset` objects and use `DataLoader` to handle batching during training and evaluation


In [3]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# Convert combined data into PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)  # Shape: (N, 2, 32, 32)
y_tensor = torch.tensor(y, dtype=torch.long)     # Labels: 0 or 1

# Perform stratified train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor, test_size=0.2, stratify=y_tensor, random_state=42
)

# Create dataset wrappers
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Define DataLoaders for mini-batch training and evaluation
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

print(f"Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")


Train size: 398400, Test size: 99600


##  Tensor Cleanup & Efficient DataLoaders

To avoid warnings and improve efficiency:
- We properly detach and clone tensors to prevent shared computation graphs.
- We re-wrap them using `TensorDataset` and define `DataLoader`s.
- We also enable multi-threaded loading (`num_workers=2`) and use `pin_memory=True` for faster GPU transfers.


In [6]:
## 🧹 Tensor Cleanup & Efficient DataLoaders

To avoid warnings and improve efficiency:
- We properly detach and clone tensors to prevent shared computation graphs.
- We re-wrap them using `TensorDataset` and define `DataLoader`s.
- We also enable multi-threaded loading (`num_workers=2`) and use `pin_memory=True` for faster GPU transfers.


##  Dataset Preparation & DataLoader Creation

We wrap the training and test tensors into `TensorDataset` objects and create `DataLoader` instances to handle batching and shuffling.
- `batch_size`: Defines the number of samples processed at once. We set it to 128 for optimal performance, but it can be increased if using a GPU.
- `shuffle=True`: Ensures the training data is shuffled to avoid learning patterns based on order.
- `shuffle


In [7]:
from torch.utils.data import TensorDataset, DataLoader

# Set batch size for training and testing
batch_size = 128  # Increase for GPU acceleration if needed

# Create datasets from tensors
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders to handle batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


##  Model Architecture: ResNet15

In this section, we define the `ResNet15` model with a custom `BasicBlock` used in the ResNet architecture. The model consists of:
- **BasicBlock**: A basic building block with two convolutional layers, batch normalization, and shortcut connections (residual connections).
- **ResNet15**: A modified version of ResNet with 15 layers, including multiple `BasicBlocks` for feature extraction and a fully connected layer at the end for classification.

#### Key components:
- **Convolutional layers**: Learn spatial features from the input data.
- **Batch Normalization**: Stabilizes and accelerates the training process.
- **Residual connections**: Help in avoiding vanishing gradients by allowing the network to learn identity mappings.


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# BasicBlock: Building block for ResNet
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut connection to match the input and output dimensions
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))  # Apply first convolution + ReLU + BatchNorm
        out = self.bn2(self.conv2(out))  # Apply second convolution + BatchNorm
        out += self.shortcut(x)  # Add shortcut connection
        out = F.relu(out)  # Final ReLU activation
        return out

# ResNet15: Main model with residual layers
class ResNet15(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet15, self).__init__()
        self.in_channels = 32

        # Initial convolution layer with batch norm
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3, stride=1, padding=1,_


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [11]:
model = ResNet15().to(device)


##  Model Training Loop

In this section, we define the training loop for our `ResNet15` model. The model undergoes training for multiple epochs, where:
- **Inputs** are fetched in batches.
- **Loss** is calculated based on the predictions of the model.
- **Backpropagation** is performed to update the model's weights using the optimizer.
- The **running loss** for each epoch is computed to monitor the progress of training.

The input shape is adjusted with `inputs.permute(0, 3, 1, 2)` to match the expected format (channels, height, width).


In [15]:
# Number of epochs for training
num_epochs = 30

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0  # Initialize running loss for the epoch

    for inputs, labels in train_loader:  # Loop through training batches
        inputs, labels = inputs.to(device), labels.to(device)  # Send data to the device (GPU/CPU)

        # Fix the input shape to match the expected format (batch_size, channels, height, width)
        inputs = inputs.permute(0, 3, 1, 2)

        optimizer.zero_grad()  # Zero the gradients to prevent accumulation from previous steps
        outputs = model(inputs)  # Perform forward pass (model prediction)
        loss = criterion(outputs, labels)  # Compute loss between model output and true labels
        loss.backward()  # Backpropagate the gradients
        optimizer.step()  # Update model parameters using the optimizer

        running_loss += loss.item()  # Accumulate loss for averaging later

    avg_loss = running_loss / len(train_loader)  # Calculate average loss for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")  # Print the loss for this epoch

print("Training Complete")  # Indicate that the training is finished

Epoch 1/30, Loss: 0.6151
Epoch 2/30, Loss: 0.5700
Epoch 3/30, Loss: 0.5585
Epoch 4/30, Loss: 0.5530
Epoch 5/30, Loss: 0.5486
Epoch 6/30, Loss: 0.5450
Epoch 7/30, Loss: 0.5418
Epoch 8/30, Loss: 0.5394
Epoch 9/30, Loss: 0.5377
Epoch 10/30, Loss: 0.5350
Epoch 11/30, Loss: 0.5331
Epoch 12/30, Loss: 0.5315
Epoch 13/30, Loss: 0.5292
Epoch 14/30, Loss: 0.5271
Epoch 15/30, Loss: 0.5251
Epoch 16/30, Loss: 0.5226
Epoch 17/30, Loss: 0.5201
Epoch 18/30, Loss: 0.5174
Epoch 19/30, Loss: 0.5145
Epoch 20/30, Loss: 0.5109
Epoch 21/30, Loss: 0.5065
Epoch 22/30, Loss: 0.5019
Epoch 23/30, Loss: 0.4962
Epoch 24/30, Loss: 0.4895
Epoch 25/30, Loss: 0.4814
Epoch 26/30, Loss: 0.4722
Epoch 27/30, Loss: 0.4618
Epoch 28/30, Loss: 0.4507
Epoch 29/30, Loss: 0.4378
Epoch 30/30, Loss: 0.4248
Training Complete


##  Model Evaluation

After training the model, we evaluate its performance on the test set. We set the model to evaluation mode with `model.eval()`, ensuring that batch normalization and dropout layers work in inference mode. 

The following steps are performed:
- **Prediction**: For each batch in the test set, the model predicts the class (photon or electron).
- **Accuracy**: The accuracy score is calculated, and a **classification report** is printed to assess precision, recall, f1-score, and support for each class.


In [17]:
# Set the model to evaluation mode (inference mode)
model.eval()
all_preds = []  # List to store predictions
all_labels = []  # List to store true labels

# Disable gradient calculation to speed up inference and save memory
with torch.no_grad():
    for inputs, labels in test_loader:  # Loop through test data
        inputs, labels = inputs.to(device), labels.to(device)  # Send data to device (GPU/CPU)
        inputs = inputs.permute(0, 3, 1, 2)  # Fix input shape [B, C, H, W]
        outputs = model(inputs)  # Perform forward pass (model prediction)
        preds = torch.argmax(outputs, dim=1)  # Get class predictions (photon or electron)
        
        # Store the predictions and true labels on the CPU (for evaluation)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy score
from sklearn.metrics import accuracy_score, classification_report

acc = accuracy_score(all_labels, all_preds)  # Compute accuracy
print(f"Test Accuracy: {acc:.4f}")

# Print detailed classification metrics
print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=["Photon", "Electron"]))

Test Accuracy: 0.7171

Classification Report:
               precision    recall  f1-score   support

      Photon       0.71      0.74      0.72     49800
    Electron       0.73      0.69      0.71     49800

    accuracy                           0.72     99600
   macro avg       0.72      0.72      0.72     99600
weighted avg       0.72      0.72      0.72     99600



##  Saving the Model

After training and evaluating the model, it is saved to disk using `torch.save()`. This allows you to load the model later for inference or further training. The model's **state dictionary** (which contains the weights and biases of the model) is saved to a `.pth` file.

In this case, we save the trained `ResNet15` model to a file named `resnet15_cms.pth`.

In [18]:
torch.save(model.state_dict(), "resnet15_cms.pth")
